In [7]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, BatchNormalization, Conv2D, Dropout, Activation, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras import regularizers, optimizers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [15]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [16]:
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

In [17]:
mean = np.mean(x_train, axis=(0, 1, 2, 3))
std = np.std(x_train, axis=(0, 1, 2, 3))
x_train = (x_train - mean)/std
x_test = (x_test - mean)/std
x_vaild = (x_valid - mean)/std